## Modeling Rush Yards Per Game 

https://www.statsmodels.org/devel/mixed_linear.html

https://www.statsmodels.org/devel/gam.html

https://pygam.readthedocs.io/en/latest/notebooks/tour_of_pygam.html

#### Modeling Framework 
    1. Fit 3 models for each type of data 
        * GAM and / or spline model - because we have panel data 
    2. Tune spline parameters 
    3. Train / test split - not cross validate 
        * work on back testing modeling framework 
    4. Output: 
        1. Prediction
        2. A notion of variance or uncertainty for optimization 
    5. Pipeline to injest new data 

In [1]:
# %load ../api_access_snippet.py
#import library
import gspread 
#Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Print nicely
import pprint
#Create scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('../quickstart/g_sheet_creds.json',scope)
#create gspread authorize using that credential
client = gspread.authorize(creds)
my_email = 'matthewjchristy66@gmail.com'

def read_file(sheet_name):
    out = client.open(sheet_name).sheet1
    out = out.get_all_values()
    out = pd.DataFrame(out, columns = out.pop(0))
    return(out)

import pandas as pd 
import numpy as np

In [2]:
rush = read_file('rushing_data_model_ready')

In [3]:
rush.head()

,Name,Week,Opp,Score,Att,Yds,TD,FUM,rush_fantasy_pts,total_fum,...,lag3_TD,lag3_rush_fantasy_pts,lag2_Yds,lag2_TD,lag2_rush_fantasy_pts,last_week_Yds,last_week_TD,last_week_rush_fantasy_pts,yds_ration,fantasy_pts_ratio
0,Marlon Mack,1,@ LAC,L 24-30,25,174.0,1.0,0.0,23.4,1,...,0.5625,10.8825,75.7,0.5625,10.8825,75.7,0.5625,10.8825,1.0,1.0
1,Marlon Mack,2,@ TEN,W 19-17,20,51.0,0.0,0.0,5.1,1,...,0.5625,10.8825,75.7,0.5625,10.8825,174.0,1.0,23.4,1.0,1.0
2,Marlon Mack,3,vs ATL,W 27-24,16,74.0,1.0,0.0,13.4,1,...,0.5625,10.8825,112.5,0.5,14.25,51.0,0.0,5.1,1.486129458388375,1.3094417643004823
3,Marlon Mack,4,vs OAK,L 24-31,11,39.0,0.0,0.0,3.9,1,...,0.6666666666666666,13.966666666666667,62.5,0.5,9.25,74.0,1.0,13.4,0.6270903010033444,0.662291169451074
4,Marlon Mack,5,@ KC,W 19-13,29,132.0,0.0,0.0,13.2,1,...,0.3333333333333333,7.466666666666666,56.5,0.5,8.649999999999999,39.0,0.0,3.9,1.0335365853658538,1.1584821428571428


In [4]:
target = ['rush_fantasy_pts']
last_week = ['last_week_Yds', 'last_week_TD', 'last_week_rush_fantasy_pts']
inputs = ['yds_ration', 'fantasy_pts_ratio', 'lag2_Yds', 'lag2_rush_fantasy_pts', 'lag2_TD']

In [5]:
rush = rush.replace('na', np.NaN);
fix_vars = target + last_week + inputs
rush[fix_vars] = rush[fix_vars].astype(float)
rush.loc[rush.rush_fantasy_pts < 0, 'rush_fantasy_pts'] = 0

In [25]:
rush['log1p_target'] = np.log1p(rush.rush_fantasy_pts + 1)
rush['yds_ration'] = rush['yds_ration'].fillna(0)
rush['fantasy_pts_ratio'] = rush['fantasy_pts_ratio'].fillna(0)
rush['yds_ration'] = rush['yds_ration'].replace([np.inf, -np.inf], 0)
rush['fantasy_pts_ratio'] = rush['fantasy_pts_ratio'].replace([np.inf, -np.inf], 0)

In [7]:
rush.dtypes

Name                           object
Week                           object
Opp                            object
Score                          object
Att                            object
Yds                            object
TD                             object
FUM                            object
rush_fantasy_pts              float64
total_fum                      object
lag3_Yds                       object
lag3_TD                        object
lag3_rush_fantasy_pts          object
lag2_Yds                      float64
lag2_TD                       float64
lag2_rush_fantasy_pts         float64
last_week_Yds                 float64
last_week_TD                  float64
last_week_rush_fantasy_pts    float64
yds_ration                    float64
fantasy_pts_ratio             float64
log1p_target                  float64
dtype: object

In [17]:
rush.isnull().any()

Name                          False
Week                          False
Opp                           False
Score                         False
Att                           False
Yds                           False
TD                            False
FUM                           False
rush_fantasy_pts              False
total_fum                     False
lag3_Yds                      False
lag3_TD                       False
lag3_rush_fantasy_pts         False
lag2_Yds                      False
lag2_TD                       False
lag2_rush_fantasy_pts         False
last_week_Yds                 False
last_week_TD                  False
last_week_rush_fantasy_pts    False
yds_ration                    False
fantasy_pts_ratio             False
log1p_target                  False
dtype: bool

## Fitting GLMs

### Fitting Mixed Model

In [11]:
#importing statsmodels packages
import statsmodels.api as sm

from statsmodels.gam.api import GLMGam, BSplines, CyclicCubicSplines

import statsmodels.formula.api as smf

In [12]:
x_list = inputs + last_week
f = 'log1p_target ~ last_week_rush_fantasy_pts + last_week_TD + last_week_Yds + yds_ration + fantasy_pts_ratio + lag2_Yds + lag2_TD + lag2_rush_fantasy_pts'

In [27]:
mixed_model = smf.mixedlm(f, data = rush, groups = rush['Name'])

In [28]:
mx_fit = mixed_model.fit()
mx_fit.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                Mixed Linear Model Regression Results
======================================================================
Model:                MixedLM     Dependent Variable:     log1p_target
No. Observations:     1223        Method:                 REML        
No. Groups:           116         Scale:                  0.3142      
Min. group size:      1           Likelihood:             -1162.4645  
Max. group size:      16          Converged:              Yes         
Mean group size:      10.5                                            
----------------------------------------------------------------------
                           Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
----------------------------------------------------------------------
Intercept                   1.534    0.056 27.384 0.000   1.424  1.644
last_week_rush_fantasy_pts  1.182    0.646  1.830 0.067  -0.084  2.448
last_week_TD               -7.167    3.879 -1.848 0.065 -14.769  0.435
last_week_Yds              -0.115    0.065 -1.790 0.073  -0.242  0.011
yds_ration                 -0.139    0.044 -3.190 0.001  -0.224 -0.054
fantasy_pts_ratio          -0.000    0.000 -0.372 0.710  -0.000  0.000
lag2_Yds                    0.020    0.048  0.420 0.674  -0.074  0.114
lag2_TD                     1.058    2.899  0.365 0.715  -4.625  6.740
lag2_rush_fantasy_pts      -0.163    0.482 -0.338 0.735  -1.108  0.782
Group Var                   0.108    0.050                            
======================================================================

"""

### Trying a Spline Regression 

In [31]:
complexity = 2
num_vars = len(x_list)

df_iter = np.repeat(complexity, num_vars)
degrees = np.repeat(complexity - 1, num_vars)

In [32]:
x_spline = rush[x_list]
bs = BSplines(x_spline, df = df_iter, degree = degrees)
cs = CyclicCubicSplines(x_spline, df = df_iter)

ValueError: 0<=der=2<=k=1 must hold

In [ ]:
pass_gam = GLMGam.from_formula(f, data = pass_mdl_data, smoother = cs, family=sm.families.Gaussian())

## Going the Bayesian Route 

https://docs.pymc.io/notebooks/GLM-linear.html

In [8]:
import pymc3 as pm
f = 'log1p_target ~ last_week_rush_fantasy_pts + last_week_TD + last_week_Yds + yds_ration + fantasy_pts_ratio + lag2_Yds + lag2_TD + lag2_rush_fantasy_pts'

In [10]:
with pm.Model() as model:
    # specify glm and pass in data. The resulting linear model, its likelihood and
    # and all its parameters are automatically added to our model.
    pm.glm.GLM.from_formula(f, rush)
    trace = pm.sample(250, cores=1) # draw 3000 posterior samples using NUTS sampling

Only 250 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [sd, lag2_rush_fantasy_pts, lag2_TD, lag2_Yds, fantasy_pts_ratio, yds_ration, last_week_Yds, last_week_TD, last_week_rush_fantasy_pts, Intercept]
  0%|          | 0/750 [00:03<?, ?it/s]


SamplingError: Bad initial energy